# Lisa4Yolo

In order to train our YOLO-V3 model(https://pjreddie.com/darknet/yolo/) with a traffic sign dataset, we need to modify the annotation file of LISA-TS into COCOdataset format. That the whole purpose of this program.

### Download
**LISA-TS:** http://cvrr.ucsd.edu/vivachallenge/index.php/signs/sign-detection/
Three dataset and the provided python toolkit

In [14]:
# Read annotation csv file
import pandas as pd
csvfile = pd.read_csv("./training/allTrainingAnnotations.csv",sep = ";")

In [15]:
csvfile.head()

,Filename,Annotation tag,Upper left corner X,Upper left corner Y,Lower right corner X,Lower right corner Y,Origin file,Origin frame number,Origin track,Origin track frame number
0,2014-04-24_10-59/frameAnnotations-cam1-2.avi_a...,doNotEnter,1071,384,1116,432,training/2014-04-24_10-59/cam1-2.avi,358,doNotEnter_1399493428.avi,1
1,2014-04-24_10-59/frameAnnotations-cam1-2.avi_a...,doNotEnter,1129,367,1182,425,training/2014-04-24_10-59/cam1-2.avi,363,doNotEnter_1399493428.avi,6
2,2014-04-24_10-59/frameAnnotations-cam1-2.avi_a...,curveRight,691,429,728,467,training/2014-04-24_10-59/cam1-2.avi,545,curveRight_1399493457.avi,1
3,2014-04-24_10-59/frameAnnotations-cam1-2.avi_a...,curveRight,703,426,743,467,training/2014-04-24_10-59/cam1-2.avi,550,curveRight_1399493457.avi,6
4,2014-04-24_10-59/frameAnnotations-cam1-2.avi_a...,curveRight,720,417,764,466,training/2014-04-24_10-59/cam1-2.avi,555,curveRight_1399493457.avi,11


In [43]:
# Extract the path to each image
pathes = csvfile['Filename']
with open("train.txt", 'w') as t:
    for i in pathes:
        t.write(i+"\n")

In [36]:
# Generate the 'obj.names'
tag = csvfile['Annotation tag']
myclass = tag.unique()
namefile = open('obj.name', 'w')
for tag in myclass:
    namefile.write(tag+"\n")
namefile.close()

In [37]:
# we also need the size of the original image
# we can get it according to the name provided
# append to column to the panda dataframe
# from PIL import Image
# for path in pathes:
#     im = Image.open("training/"+path);
#     print(im.size)

**We actually found that all the pictures are 1280 * 960 from the code above**

In [42]:
# we need to create a list of <object-class> <x_center> <y_center> <width> <height>
# so first we need to create a hashmap that map object-class to its number
index = 0
class2index = {}
for i in myclass:
    class2index[i] = index
    index += 1
print(class2index)

{'doNotEnter': 0, 'curveRight': 1, 'speedLimit35': 2, 'pedestrianCrossing': 3, 'stop': 4, 'stopAhead': 5, 'signalAhead': 6, 'intersectionLaneControl': 7, 'bicyclesMayUseFullLane': 8, 'speedLimit25': 9, 'yieldToPedestrian': 10, 'noParking': 11, 'speedLimit15': 12, 'speedBumpsAhead': 13, 'speedLimit30': 14, 'curveLeft': 15, 'noLeftAndUTurn': 16, 'noUTurn': 17, 'keepRight': 18, 'leftAndUTurnControl': 19, 'merge': 20, 'speedLimit40': 21, 'speedLimit45': 22, 'addedLane': 23, 'speedLimit50': 24, 'school': 25, 'intersection': 26, 'laneEnds': 27, 'noRightTurn': 28, 'yieldAhead': 29, 'speedLimit60': 30}


In [89]:
for item in csvfile.iterrows():
    name = item[1][0]
    arr = name.split("/")
    path = arr[0] + "/" + arr[1]
    filename = arr[2]
    fn = arr[2].split(".")
    filename = "training/" + path + "/" + fn[0]+"." + fn[1] + ".txt"
    #for example: <x> = <absolute_x> / <image_width> or <height> = <absolute_height> / <image_height>
    #atention: <x_center> <y_center> - are center of rectangle (are not top-left corner)
    row = (str)(class2index[item[1][1]]) + " "
    x_center = round((item[1][2] + item[1][4])/2,6)
    y_center = round((item[1][3] + item[1][5])/2,6)
    width = round(abs(item[1][2] - item[1][4])/1280,6)
    height = round(abs(item[1][3] - item[1][5])/960,6)
    row = row + str(x_center) + " " + str(y_center) + " " + str(width) + " " + str(height)
    # Now we have everything, time to write the file
    f = open(filename, 'w')
    f.write(row)
    f.close